# ETL of the CIC IoT 2023 Dataset for Cybersecurity Research

[University of New Brunswick - Canadian Institute for Cybersecurity](https://www.unb.ca/cic/datasets/index.html)

This notebook consolidates all the code needed to download, extract, transform and load the dataset from CIC IoT 2023.

# Imports

In [1]:
import os
import sys

import pandas as pd

sys.path.append('../') 

## Definitions

In [2]:
from utils import get_constants

constants = get_constants()

features = constants['features']
path = constants['path']
csv_path = constants['csv_path']
parquet_path = constants['parquet_path']
attack_category_map = constants['attack_category_map']

# Ingestion

## Download dataset

In [3]:
# !wget -P {path} http://205.174.165.80/IOTDataset/CIC_IOT_Dataset2023/Dataset/CSV/CICIoT2023.zip
!du -sh {path + 'CICIoT2023.zip'}

2,7G	/var/fasttmp/dsn/CICIoT2023.zip


In [26]:
# !unzip {constants['path'] + 'CICIoT2023.zip'} -d {constants['csv_path']}
!du -sh {csv_path}
!echo "CSV files: $(ls -1q {csv_path} | wc -l)"

13G	/var/fasttmp/dsn/unb_cic_csv
CSV files: 169


The dataset comes as a zip of 169 CSV files. The zipped file is 2.7GB in size, and the extracted CSVs add to 13GB in disk, as we can see above.

## Load and Transform

In [5]:
columns_dtype = {
    'category': [
        'label'
    ],
    'bool': [
        *features['protocol'],
        *features['tcp_flag']
    ],
    'float32': [
        *features['tcp_flag_counts'],
        *features['flow'],
        *features['packet'],
    ],
}

column_dtype_map = {
    col: dtype
    for dtype, column_list in columns_dtype.items()
    for col in column_list
}

Since most of the floating point features are related to a simple average of the pre-defined windows inside a flow, there isn't much to gain having a double precision here, so we're going to work with `float32` for all floating point values.

In [6]:
%%time
csv_files = (
    filename
    for filename in sorted(os.listdir(csv_path))
    if filename.endswith('.csv')
)

df = pd.concat(
    pd.read_csv(os.path.join(csv_path, csv), index_col=None, header=0, dtype=column_dtype_map)
    for csv in csv_files
)

df['general_label'] = df['label'].map(attack_category_map).astype('category')

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 46686579 entries, 0 to 234744
Data columns (total 48 columns):
 #   Column           Dtype   
---  ------           -----   
 0   flow_duration    float32 
 1   Header_Length    float32 
 2   Protocol Type    float32 
 3   Duration         float32 
 4   Rate             float32 
 5   Srate            float32 
 6   Drate            float32 
 7   fin_flag_number  bool    
 8   syn_flag_number  bool    
 9   rst_flag_number  bool    
 10  psh_flag_number  bool    
 11  ack_flag_number  bool    
 12  ece_flag_number  bool    
 13  cwr_flag_number  bool    
 14  ack_count        float32 
 15  syn_count        float32 
 16  fin_count        float32 
 17  urg_count        float32 
 18  rst_count        float32 
 19  HTTP             bool    
 20  HTTPS            bool    
 21  DNS              bool    
 22  Telnet           bool    
 23  SMTP             bool    
 24  SSH              bool    
 25  IRC              bool    
 26  TCP              bo

We can see that some features names follow different patterns, but thinking about the consistency with the original dataset, we chose to keep the original names for this work.

## Save

In [27]:
%%time
sort_columns = ['general_label', 'label', 'Protocol Type', 'Tot size', 'Header_Length']

df.sort_values(sort_columns).to_parquet(parquet_path, index=False)

!du -sh {parquet_path}

960M	/var/fasttmp/dsn/unb_cic_ds.parquet
CPU times: user 1min 53s, sys: 5.07 s, total: 1min 59s
Wall time: 1min 46s


Here we're converting the CSVs to a parquet to make it easier to work with the data.

We already define the schema and save the dataset with the values sorted to take advantage of some optimizations related to disk space usage, resulting in a dataset with less than 1GB in disk.

In [8]:
# del df

In [9]:
# !rm -r {csv_path}